In [1]:
import numpy as np
import pandas as pd

In [3]:
from model.calculate_features import CalculateFeatures

dtypes = {'Magnitude': np.float64, 'Latitude': np.float64, 'Longitude': np.float64, 'Depth': np.float64}
df = pd.read_csv('../../data/DATA_2.csv', delimiter=',', dtype=dtypes,parse_dates=["Datetime"],date_parser=np.datetime64)
df = df[:16]
a=CalculateFeatures(df,3)

In [4]:
a.features

,firstT,lastT,meanMag,a,b,elapsedT,rateSqrtEnergy,meanTDiff,maxMag
0,1988-01-01 05:42:07,1988-01-01 15:55:44,4.233333,2.356720,0.492494,36817,112300.945105,18408.5,4.6
1,1988-01-01 15:26:55,1988-01-01 20:22:04,4.366667,5.813672,1.273888,17709,265077.702018,8854.5,4.6
2,1988-01-01 15:55:44,1988-01-01 20:52:15,4.266667,3.731581,0.814397,17791,233548.863335,8895.5,4.6
3,1988-01-01 20:22:04,1988-01-02 00:19:04,4.066667,10.019546,2.385606,14220,190624.115594,7110.0,4.2
4,1988-01-01 20:52:15,1988-01-02 03:39:45,4.066667,10.019546,2.385606,24450,110866.050051,12225.0,4.2
5,1988-01-02 00:19:04,1988-01-02 04:15:05,4.100000,9.144958,2.165321,14161,201991.919025,7080.5,4.2
6,1988-01-02 03:39:45,1988-01-02 09:35:41,4.033333,3.897477,0.898260,21356,123076.449512,10678.0,4.2
7,1988-01-02 04:15:05,1988-01-02 16:04:46,3.900000,6.520657,1.590404,42581,48583.723413,21290.5,4.1
8,1988-01-02 09:35:41,1988-01-02 21:19:35,3.866667,9.542425,2.385606,42234,45437.583111,21117.0,4.0
9,1988-01-02 16:04:46,1988-01-03 03:14:30,4.100000,3.194428,0.717332,40184,80637.035870,20092.0,4.5
